In [21]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import spacy
import re
import pickle
from inltk.inltk import tokenize
from time import time

from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
from torchtext import data
from torchtext import datasets

from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
from torchtext.datasets import TranslationDataset
from torchtext.data import Field, BucketIterator

In [22]:
spacy_eng = spacy.load("en_core_web_sm")
str_punct = '''[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~।]'''

def tokenize_hi(text):
    text = re.sub(str_punct,'',text).lower()
    return [tok.lower() for tok in tokenize(text, "hi")]


def tokenize_eng(text):
    text = re.sub(str_punct,'',text).lower()
    return [tok for tok in spacy_eng.tokenizer(text)]

In [23]:
hindi = Field(tokenize=tokenize_hi, lower=True, init_token="<sos>", eos_token="<eos>")
english = Field(
    tokenize=tokenize_eng, lower=True, init_token="<sos>", eos_token="<eos>"
)

hindi = data.Field(tokenize=tokenize_hi)
english = data.Field(tokenize=tokenize_eng)
mt_train = datasets.TranslationDataset(
     path='./data_torch/data_sm', exts=('.hi', '.en'),
     fields=(hindi, english))
hindi.build_vocab(mt_train, max_size=15000, min_freq=2)
english.build_vocab(mt_train, max_size=15000, min_freq=2)

# with open('hindi_vocab.pickle', 'wb') as handle:
#     pickle.dump(hindi, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('english_vocab.pickle', 'wb') as handle:
#     pickle.dump(english, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out

In [25]:
# We're ready to define everything we need for training our Seq2Seq model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_model = True
save_model = True

# Training hyperparameters
num_epochs = 1000
learning_rate = 3e-4
batch_size = 32

# Model hyperparameters
src_vocab_size = len(hindi.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi["<pad>"]

# Tensorboard to get nice loss plot
# writer = SummaryWriter("runs/loss_plot")
step = 0

print(f'length of src_vocab_size is {src_vocab_size}')

print(f'length of input_size_decoder is {trg_vocab_size}')


train_iterator = data.BucketIterator(
     dataset=mt_train, batch_size=batch_size,
     sort_key=lambda x: data.interleave_keys(len(x.src), len(x.trg)), device=device)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)

pad_idx = 1
# english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

sentence = "प्रधानमंत्री ने कहा कि भारत में केंद्र सरकार बुनियादी ढांचे पर ध्यान केंद्रित कर रही है।"

length of src_vocab_size is 120
length of input_size_decoder is 2


In [26]:
################################################################################################################################################

import torch
import spacy
from torchtext.data.metrics import bleu_score
import sys


def translate_sentence(model, sentence, german, english, device, max_length=50):
    # Load german tokenizer
    str_punct = '''[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~।]'''
    
    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
#         tokens = [token.text.lower() for token in spacy_ger(sentence)]
        sentence = re.sub(str_punct,'',sentence).lower()
        tokens = [re.sub(str_punct,'',tok).lower() for tok in tokenize(sentence, "hi")]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]


def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

################################################################################################################################################

In [27]:
sentence = 'Our ancient texts also say – “Mahajaney yen gatah panthaya” – it is beneficial to walk the path on which Mahajans walk'

In [28]:
mt_train[0].__dict__.values()

dict_values([['▁अग्रिम', '▁धन', '▁राशि', '▁इन', '▁अस्पतालों', '▁को', '▁चिकित्सा', '▁निरीक्षक', 'ों', '▁को', '▁दी', '▁जाएगी', '▁जो', '▁हर', '▁मामले', '▁को', '▁देखते', '▁हुए', '▁सहायता', '▁प्रदान', '▁करेंगे'], [an, advance, is, placed, with, the, medical, superintendents, of, such, hospitals, who, then, provide, assistance, on, a, case, to, case, basis]])

In [30]:
for epoch in range(num_epochs):
    st = time()

    print(f"[Epoch {epoch} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
#         save_checkpoint(checkpoint)

    model.eval()
    translated_sentence = translate_sentence(
        model, sentence, hindi, english, device, max_length=50
    )

    print(f"Translated example sentence: \n {translated_sentence}")
    model.train()
    losses = []

    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # Forward prop
        output = model(inp_data, target[:-1, :])

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin.
        # Let's also remove the start token while we're at it
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(output, target)
        losses.append(loss.item())

        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # plot to tensorboard
#         writer.add_scalar("Training loss", loss, global_step=step)
        step += 1
    print(f'Total time taken for the epoch number {epoch} was {time() - st}')


    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)

[Epoch 0 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 0 was 0.2482149600982666
[Epoch 1 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 1 was 0.24390935897827148
[Epoch 2 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 2 was 0.22967195510864258
[Epoch 3 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 3 was 0.22107911109924316
[Epoch 4 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 4 was 0.2182447910308838
[Epoch 5 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 5 was 0.21697044372558594
[Epoch 6 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 6 was 0.21791553497314453
[Epoch 7 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 7 was 0.21593642234802246
[Epoch 8 / 1000]
Translate

Total time taken for the epoch number 66 was 0.2164449691772461
[Epoch 67 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 67 was 0.22393512725830078
[Epoch 68 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 68 was 0.2169475555419922
[Epoch 69 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 69 was 0.22028064727783203
[Epoch 70 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 70 was 0.21674871444702148
[Epoch 71 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 71 was 0.21790194511413574
[Epoch 72 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 72 was 0.21762728691101074
[Epoch 73 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 73 was 0.21756315231323242
[Epoch 74 / 1000]
Translated example sentence: 
 ['<unk>']
Total time

Total time taken for the epoch number 132 was 0.2189319133758545
[Epoch 133 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 133 was 0.21713805198669434
[Epoch 134 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 134 was 0.21681451797485352
[Epoch 135 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 135 was 0.21912884712219238
Epoch   136: reducing learning rate of group 0 to 3.0000e-05.
[Epoch 136 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 136 was 0.21798419952392578
[Epoch 137 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 137 was 0.2169630527496338
[Epoch 138 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 138 was 0.21624183654785156
[Epoch 139 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 139 was 0.2171621322

Total time taken for the epoch number 195 was 0.21857404708862305
[Epoch 196 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 196 was 0.21817612648010254
[Epoch 197 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 197 was 0.22060799598693848
[Epoch 198 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 198 was 0.2172229290008545
[Epoch 199 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 199 was 0.2177734375
[Epoch 200 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 200 was 0.2188563346862793
[Epoch 201 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 201 was 0.21801972389221191
[Epoch 202 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 202 was 0.21751976013183594
[Epoch 203 / 1000]
Translated example sentence: 
 ['<unk>']
T

Total time taken for the epoch number 261 was 0.22188568115234375
[Epoch 262 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 262 was 0.21686339378356934
[Epoch 263 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 263 was 0.21678900718688965
[Epoch 264 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 264 was 0.2166271209716797
[Epoch 265 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 265 was 0.21794605255126953
[Epoch 266 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 266 was 0.21722006797790527
[Epoch 267 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 267 was 0.21687746047973633
[Epoch 268 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 268 was 0.21750593185424805
[Epoch 269 / 1000]
Translated example sentence: 
 ['<

...

Total time taken for the epoch number 723 was 0.21787333488464355
[Epoch 724 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 724 was 0.21636343002319336
[Epoch 725 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 725 was 0.21705222129821777
[Epoch 726 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 726 was 0.21684932708740234
[Epoch 727 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 727 was 0.2167830467224121
[Epoch 728 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 728 was 0.21683239936828613
[Epoch 729 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 729 was 0.21747064590454102
[Epoch 730 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 730 was 0.21921634674072266
[Epoch 731 / 1000]
Translated example sentence: 
 ['<

Total time taken for the epoch number 789 was 0.21695947647094727
[Epoch 790 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 790 was 0.218170166015625
[Epoch 791 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 791 was 0.21631956100463867
[Epoch 792 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 792 was 0.21775007247924805
[Epoch 793 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 793 was 0.21917295455932617
[Epoch 794 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 794 was 0.21925854682922363
[Epoch 795 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 795 was 0.21770787239074707
[Epoch 796 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 796 was 0.21695423126220703
[Epoch 797 / 1000]
Translated example sentence: 
 ['<u

Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 855 was 0.2201070785522461
[Epoch 856 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 856 was 0.21917033195495605
[Epoch 857 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 857 was 0.22405290603637695
[Epoch 858 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 858 was 0.21836328506469727
[Epoch 859 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 859 was 0.21942567825317383
[Epoch 860 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 860 was 0.21885466575622559
[Epoch 861 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 861 was 0.2179405689239502
[Epoch 862 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 862 was 0.2179126739501953
[Epoch 863 / 1

Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 921 was 0.21758747100830078
[Epoch 922 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 922 was 0.2181229591369629
[Epoch 923 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 923 was 0.22420954704284668
[Epoch 924 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 924 was 0.21893978118896484
[Epoch 925 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 925 was 0.21795129776000977
[Epoch 926 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 926 was 0.21828699111938477
[Epoch 927 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 927 was 0.21918916702270508
[Epoch 928 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 928 was 0.21861863136291504
[Epoch 929 /

Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 987 was 0.22171783447265625
[Epoch 988 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 988 was 0.21938800811767578
[Epoch 989 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 989 was 0.22340679168701172
[Epoch 990 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 990 was 0.22192120552062988
[Epoch 991 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 991 was 0.21745777130126953
[Epoch 992 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 992 was 0.21758770942687988
[Epoch 993 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 993 was 0.22134041786193848
[Epoch 994 / 1000]
Translated example sentence: 
 ['<unk>']
Total time taken for the epoch number 994 was 0.21867895126342773
[Epoch 995 